In [0]:
import pandas as pd
import numpy as np
import chess
import re
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import io
import warnings
from sklearn.ensemble import GradientBoostingRegressor as xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
import lightgbm as lgb
import h2o
from sklearn.ensemble import ExtraTreesRegressor
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimatorfrom sklearn.ensemble import GradientBoostingRegressor as xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
import lightgbm as lgb
import h2o
from sklearn.ensemble import ExtraTreesRegressor
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator

warnings.filterwarnings('ignore')

In [0]:
!pip install h2o

In [0]:
pip install python-chess~=0.26

In [0]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import chess
import chess.pgn
import chess.engine

# PGN to CSV


# Raw Data Generation


In [0]:
raw_data = pd.read_csv('/content/gdrive/My Drive/cse519_project/dataset.csv')

In [0]:
raw_data_2 = pd.read_csv('/content/gdrive/My Drive/cse519_project/dataset_2019.csv')
len(raw_data_2)

4324

In [0]:
rob_data = pd.read_csv('/content/gdrive/My Drive/cse519_project/dataset_rob.csv').iloc[:16000]

# Feature Engineer'

## ELO part

In [0]:
data = pd.concat([raw_data, raw_data_2])

In [0]:
def rob_transcript(record):
  target = "{[+-]\d{1,2}.\d{1,2}/\d{1,2}[ ]?\d{1,2}s}"
  match1 = re.findall(target,record)
  match2 = re.findall("\d\.",record)
  match_obj = match1 + match2
  for m in match_obj:
    record = record.replace(m,'')
  return record
rob_data['TRANSCRIPT'] = rob_data['RECORD'].apply(lambda x : rob_transcript(x))

In [0]:
def generate_clock_list(record, return_white=True):
  target = "\d{1,2}:\d{1,2}:\d{1,2}"
  match_obj = re.findall(target, record)
  white_clock = []
  black_clock = []
  for i in range(len(match_obj)):
    if i%2 == 0:
      white_clock.append(match_obj[i])
    else:
      black_clock.append(match_obj[i])
 
  if return_white:
    return white_clock
  else:
    return black_clock

In [0]:
def generate_eval_list(record, return_white=True):
  target = "eval\s\#?[-+]?\d{1,2}\.?\d{1,2}"
  match_eval = re.findall(target, record)
  white_eval = []
  black_eval = []
  for i in range(len(match_eval)):
    try:
      local_eval = float(match_eval[i][5:].strip('#'))
    except:
      local_eval = float(match_eval[i][5:])
    if i%2 == 0:
      white_eval.append(local_eval)
    else:
      black_eval.append(local_eval)
  if return_white:
    return white_eval
  else:
    return black_eval

In [0]:
data['WHITE_LOCAL_EVAL'] = data['RECORD'].apply(lambda x : generate_eval_list(x,True))
data['BLACK_LOCAL_EVAL'] = data['RECORD'].apply(lambda x : generate_eval_list(x,False))

In [0]:
def generate_clock_list(record, return_white=True):
  target = "\d{1,2}:\d{1,2}:\d{1,2}"
  match_obj = re.findall(target, record)
  white_clock = []
  black_clock = []
  for i in range(len(match_obj)):
    if i%2 == 0:
      white_clock.append(match_obj[i])
    else:
      black_clock.append(match_obj[i])
 
  if return_white:
    return white_clock
  else:
    return black_clock

In [0]:
data['WHITE_CLOCK'] = data['RECORD'].apply(lambda x : generate_clock_list(x,True))
data['BLACK_CLOCK'] = data['RECORD'].apply(lambda x : generate_clock_list(x,False))

In [0]:
def generate_position_list(record):
  match1 = re.findall("\s\{\s\[%eval\s\#?[-+]?[0-9]*\.?[0-9]+\]\s\[%clk\s\d{1,2}:\d{1,2}:\d{1,2}\]\s\}",record)
  match2 = re.findall("\d{2}\.",record)
  match3 = re.findall("\d\.",record)
  match = match1 + match2 + match3

  for m in match:
    record = record.replace(m,'')
  record = record.replace('..','').split(' ')
  record = [elmt for elmt in record if elmt != '' and elmt[0].isalpha()]
  return record

In [0]:
data['TRANSCRIPT'] = data['RECORD'].apply(lambda x : generate_position_list(x))

In [0]:
def generate_final_location(transcript, return_matrix=False):
  board = chess.Board()
  for elmt in transcript:
    try:
      elmt = elmt.strip('?!')
    except:
      pass
    board.push_san(elmt)
  if return_matrix:
    board_matrix = np.asarray(str(board).replace('\n',' ').split(' ')).reshape(8,8)
    return board_matrix
  else:
    return board


In [0]:
# fianl position
# check number 
# special variant-specific win
# checkmate
# stalemate
# insufficient material
# repetition

def rob_properties(transcript, property):
  b = chess.Board()
  check = 0
  checkmate = 0
  stalemate = 0
  repetition = 0
  insufficient_material = 0
  for elmt in transcript:
    try:
      b.push_san(elmt)
    except:
      continue
    # check num for both side
    if (b.was_into_check() is True) or (b.is_check() is True):
      check = check + 1
    if b.is_checkmate():
      checkmate = checkmate + 1
    if b.is_stalemate():
      stalemate = stalemate + 1
    if repetition==0:
      if b.can_claim_threefold_repetition():
        repetition = 1
    if b.is_insufficient_material():
      insufficient_material = insufficient_material + 1
  if b.is_variant_win() is True:
    variant_win = 1
  else:
    variant_win = 0
  if property is 'check':
    return check
  elif property is 'variant_win':
    return variant_win
  elif property is 'checkmate':
    return checkmate
  elif property is 'insufficient_material':
    return insufficient_material
  elif property is 'repetition':
    return repetition
  elif property is 'stalemate':
    return stalemate
  elif property is 'FINAL_POSITION_MATRIX':
    board_matrix = np.asarray(str(b).replace('\n',' ').split(' ')).reshape(8,8)
    return board_matrix


for property in ['check', 'variant_win', 'checkmate', 'insufficient_material', 'repetition', 'stalemate','FINAL_POSITION_MATRIX']:
  rob_data[property] = rob_data['TRANSCRIPT'].apply(lambda x : rob_properties(x, property=property))
  

In [0]:
data['FINAL_POSITION'] = data['TRANSCRIPT'].apply(lambda x : generate_final_location(x))

In [0]:
data['FINAL_POSITION_MATRIX'] = data['TRANSCRIPT'].apply(lambda x : generate_final_location(x,True))

# Filter data


In [0]:
# drop dataset if they do not satisfy 3 plys
data['PLYS'] = data['TRANSCRIPT'].apply(lambda x: len(x))
data_f = data[data['PLYS']>6]
# drop dataset if time_control is '-'
data_f = data_f[data_f['TIME_CONTROL']!='-']

In [0]:
# drop dataset if they do not satisfy 3 plys
rob_data['PLYS'] = rob_data['TRANSCRIPT'].apply(lambda x: len(x))
rob_data = rob_data[rob_data['PLYS']>6]

In [0]:
data_f.columns

Index(['GAME_TYPE', 'WHITE_NAME', 'BLACK_NAME', 'RESULT', 'UTC_DATE',
       'UTC_TIME', 'WHITE_ELO', 'BLACK_ELO', 'WHITE_RATING_DIFF',
       'BLACK_RATING_DIFF', 'ECO', 'OPENING', 'TIME_CONTROL', 'TERMINATION',
       'RECORD', 'WHITE_LOCAL_EVAL', 'BLACK_LOCAL_EVAL', 'WHITE_CLOCK',
       'BLACK_CLOCK', 'TRANSCRIPT', 'FINAL_POSITION', 'FINAL_POSITION_MATRIX',
       'PLYS'],
      dtype='object')

In [0]:
rob_data.columns

Index(['RESULT', 'WHITE_ELO', 'BLACK_ELO', 'RECORD', 'TRANSCRIPT', 'PLYS'], dtype='object')

# Extract Feature

In [0]:
data_f['FIXED_TIME'] = 0
data_f['ALL_TIME'] = 0
data_f['INCREMENT'] = 0
for i in range(len(data_f)):
  time_control= data_f.iloc[i]['TIME_CONTROL'].split('+')
  plys = data_f.iloc[i]['PLYS']
  try:
    fixed_time = int(time_control[0])
    increment = int(time_control[1])
    all_time = fixed_time + plys*increment

    data_f['FIXED_TIME'].iloc[i] = fixed_time
    data_f['INCREMENT'].iloc[i] = increment
    data_f['ALL_TIME'].iloc[i] = all_time
  except:
    data_f['FIXED_TIME'].iloc[i] = np.NaN
    data_f['INCREMENT'].iloc[i] = np.NaN
    data_f['ALL_TIME'].iloc[i] = np.NaN

In [0]:
# capture and captured
data_f['WHITE_CAPTURE'] = 0
data_f['WHITE_CAPTURED'] = 0
data_f['BLACK_CAPTURE'] = 0
data_f['BLACK_CAPTURED'] = 0
for i in range(len(data_f)):
  t_white = data_f.iloc[i]['TRANSCRIPT'][::2]
  t_black = data_f.iloc[i]['TRANSCRIPT'][1::2]
  # white
  count = 0
  for t in t_white:
    if 'x' in t:
      count = count + 1
  data_f['WHITE_CAPTURE'].iloc[i] = count
  data_f['BLACK_CAPTURED'].iloc[i]= count
  # black
  count = 0
  for t in t_black:
    if 'x' in t:
      count = count + 1
  data_f['WHITE_CAPTURED'].iloc[i] = count
  data_f['BLACK_CAPTURED'].iloc[i] = count

In [0]:
# rob capture and captured
rob_data['WHITE_CAPTURE'] = 0
rob_data['WHITE_CAPTURED'] = 0
rob_data['BLACK_CAPTURE'] = 0
rob_data['BLACK_CAPTURED'] = 0
for i in range(len(rob_data)):
  t_white = rob_data.iloc[i]['TRANSCRIPT'][::2]
  t_black = rob_data.iloc[i]['TRANSCRIPT'][1::2]
  # white
  count = 0
  for t in t_white:
    if 'x' in t:
      count = count + 1
  rob_data['WHITE_CAPTURE'].iloc[i] = count
  rob_data['BLACK_CAPTURED'].iloc[i]= count
  # black
  count = 0
  for t in t_black:
    if 'x' in t:
      count = count + 1
  rob_data['WHITE_CAPTURED'].iloc[i] = count
  rob_data['BLACK_CAPTURED'].iloc[i] = count

In [0]:
# number of each material left
material_num = {'Q':0,'N':0,'B':0,'R':0,'P':0,'q':0,'n':0,'b':0,'r':0,'p':0}
for key in material_num.keys():
  data_f[key] = 0

for sample in range(len(data_f)):
  material_num = {'Q':0,'N':0,'B':0,'R':0,'P':0,'q':0,'n':0,'b':0,'r':0,'p':0}
  final_position_matrix = data_f['FINAL_POSITION_MATRIX'].iloc[sample]
  for i in range(final_position_matrix.shape[0]):
    for j in range(final_position_matrix.shape[1]):
      material = final_position_matrix[i][j]
      if material == 'K' or material == 'k':
        continue
      if material.isalpha():
        material_num[material] = material_num[material] + 1
  for key in material_num.keys():
    data_f[key].iloc[sample] = material_num[key]


In [0]:
# rob number of each material left
material_num = {'Q':0,'N':0,'B':0,'R':0,'P':0,'q':0,'n':0,'b':0,'r':0,'p':0}
for key in material_num.keys():
  rob_data[key] = 0

for sample in range(len(rob_data)):
  material_num = {'Q':0,'N':0,'B':0,'R':0,'P':0,'q':0,'n':0,'b':0,'r':0,'p':0}
  final_position_matrix = rob_data['FINAL_POSITION_MATRIX'].iloc[sample]
  for i in range(final_position_matrix.shape[0]):
    for j in range(final_position_matrix.shape[1]):
      material = final_position_matrix[i][j]
      if material == 'K' or material == 'k':
        continue
      if material.isalpha():
        material_num[material] = material_num[material] + 1
  for key in material_num.keys():
    rob_data[key].iloc[sample] = material_num[key]


In [0]:
# check number 
# special variant-specific win
# checkmate
# stalemate
# insufficient material
# repetition

def record_properties(record, property=False):
  p = io.StringIO(record)
  b = chess.Board()
  game = chess.pgn.read_game(p)
  check = 0
  checkmate = 0
  stalemate = 0
  repetition = 0
  insufficient_material = 0
  for move in game.mainline_moves():
    b.push(move)
    # check num for both side
    if (b.was_into_check() is True) or (b.is_check() is True):
      check = check + 1
    if b.is_checkmate():
      checkmate = checkmate + 1
    if b.is_stalemate():
      stalemate = stalemate + 1
    if repetition==0:
      if b.can_claim_threefold_repetition():
        repetition = 1
    if b.is_insufficient_material():
      insufficient_material = insufficient_material + 1
  if b.is_variant_win() is True:
    variant_win = 1
  else:
    variant_win = 0
  if property is 'check':
    return check
  elif property is 'variant_win':
    return variant_win
  elif property is 'checkmate':
    return checkmate
  elif property is 'insufficient_material':
    return insufficient_material
  elif property is 'repetition':
    return repetition
  elif property is 'stalemate':
    return stalemate


for property in ['check', 'variant_win', 'checkmate', 'insufficient_material', 'repetition', 'stalemate']:
  data_f[property] = data_f['RECORD'].apply(lambda x : record_properties(x, property=property))


In [0]:
data_white = data_f[['GAME_TYPE', 'RESULT', 'UTC_DATE',
       'UTC_TIME', 'WHITE_ELO', 'WHITE_RATING_DIFF',
       'ECO', 'OPENING', 'TIME_CONTROL', 'TERMINATION',
       'RECORD', 'WHITE_LOCAL_EVAL',
       'WHITE_CLOCK',  'TRANSCRIPT', 'FINAL_POSITION',
       'FINAL_POSITION_MATRIX','FIXED_TIME','INCREMENT','ALL_TIME','WHITE_CAPTURE', 'WHITE_CAPTURED',
       'Q', 'N', 'B', 'R', 'P', 'q', 'n', 'b', 'r', 'p','check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate']]
data_black = data_f[['GAME_TYPE',  'RESULT', 'UTC_DATE',
       'UTC_TIME', 'BLACK_ELO', 
       'BLACK_RATING_DIFF', 'ECO', 'OPENING', 'TIME_CONTROL', 'TERMINATION',
       'RECORD',  'BLACK_LOCAL_EVAL',
        'BLACK_CLOCK', 'TRANSCRIPT', 'FINAL_POSITION',
       'FINAL_POSITION_MATRIX','FIXED_TIME','INCREMENT','ALL_TIME','BLACK_CAPTURE', 'BLACK_CAPTURED',
       'Q', 'N', 'B', 'R', 'P', 'q', 'n', 'b', 'r', 'p','check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate']]
data_white.columns = ['GAME_TYPE', 'RESULT', 'UTC_DATE', 'UTC_TIME',
       'ELO', 'RATING_DIFF', 'ECO', 'OPENING', 'TIME_CONTROL',
       'TERMINATION', 'RECORD', 'LOCAL_EVAL', 'CLOCK',
       'TRANSCRIPT_ALL', 'FINAL_POSITION', 'FINAL_POSITION_MATRIX','FIXED_TIME',
       'INCREMENT','ALL_TIME','CAPTURE','CAPTURED',
       'Q', 'N', 'B', 'R', 'P', 'q', 'n', 'b', 'r', 'p','check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate']
data_black.columns = ['GAME_TYPE', 'RESULT', 'UTC_DATE', 'UTC_TIME',
       'ELO', 'RATING_DIFF', 'ECO', 'OPENING', 'TIME_CONTROL',
       'TERMINATION', 'RECORD', 'LOCAL_EVAL', 'CLOCK',
       'TRANSCRIPT_ALL', 'FINAL_POSITION', 'FINAL_POSITION_MATRIX','FIXED_TIME','INCREMENT','ALL_TIME','CAPTURE','CAPTURED',
       'Q', 'N', 'B', 'R', 'P', 'q', 'n', 'b', 'r', 'p','check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate']


In [0]:
rob_data_white = rob_data[['RESULT', 'WHITE_ELO', 'RECORD', 
       'TRANSCRIPT', 'check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate',
       'FINAL_POSITION_MATRIX', 'WHITE_CAPTURE', 'WHITE_CAPTURED',
        'Q', 'N', 'B', 'R', 'P', 'q', 'n',
       'b', 'r', 'p']]
rob_data_black = rob_data[['RESULT', 'BLACK_ELO', 'RECORD', 
       'TRANSCRIPT', 'check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate',
       'FINAL_POSITION_MATRIX',
       'BLACK_CAPTURE', 'BLACK_CAPTURED', 'Q', 'N', 'B', 'R', 'P', 'q', 'n',
       'b', 'r', 'p']]
rob_data_white.columns = ['RESULT', 'ELO', 'RECORD', 
       'TRANSCRIPT', 'check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate',
       'FINAL_POSITION_MATRIX', 'CAPTURE', 'CAPTURED',
        'Q', 'N', 'B', 'R', 'P', 'q', 'n',
       'b', 'r', 'p']
rob_data_black.columns = ['RESULT', 'ELO', 'RECORD', 
       'TRANSCRIPT', 'check', 'variant_win', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate',
       'FINAL_POSITION_MATRIX',  'CAPTURE', 'CAPTURED',
        'Q', 'N', 'B', 'R', 'P', 'q', 'n',
       'b', 'r', 'p']

In [0]:
def rob_clock_list(record, white=True):
  target = "[+-]\d{1,2}.\d{1,2}/\d{1,2}"
  match_obj = re.findall(target, record)
  for m in match_obj:
    record = record.replace(m,'')
  target = "\d{1,2}s"
  match_obj = re.findall(target, record)
  match_obj = [int(m.strip('s')) for m in match_obj]
  if white:
    return match_obj[::2]
  else:
    return match_obj[1::2]

rob_data_white['TIME_PER_STEP'] = rob_data['RECORD'].apply(lambda x: rob_clock_list(x,True))
rob_data_black['TIME_PER_STEP'] = rob_data['RECORD'].apply(lambda x: rob_clock_list(x,False))


In [0]:
def rob_eval_list(record, white=True):
  target = "[+-]\d{1,2}.\d{1,2}/"
  match_obj = re.findall(target, record)
  obj = [float(m.strip('/')) for m in match_obj]  
  if white:
    return obj[::2]
  else:
    return obj[1::2]

rob_data_white['LOCAL_EVAL'] = rob_data['RECORD'].apply(lambda x: rob_eval_list(x,True))
rob_data_black['LOCAL_EVAL'] = rob_data['RECORD'].apply(lambda x: rob_eval_list(x,False))

In [0]:
def result(result_string, white=True):
  if white:
    try:
      result_num = int(result_string.split('-')[0])
    except:
      result_num = 0.5
    return result_num
  else:
    try:
      result_num = int(result_string.split('-')[1])
    except:
      result_num = 0.5
    return result_num

data_white['RESULT'] = data_white['RESULT'].apply(lambda x :result(x,True))
data_black['RESULT'] = data_white['RESULT'].apply(lambda x :result(x,False))

In [0]:
def result(result_string, white=True):
  if white:
    try:
      result_num = int(result_string.split('-')[0])
    except:
      result_num = 0.5
    return result_num
  else:
    try:
      result_num = int(result_string.split('-')[1])
    except:
      result_num = 0.5
    return result_num
data['RESULT'] = data['RESULT'].apply(lambda x : result(x))

rob_data_white['RESULT'] = rob_data_white['RESULT'].apply(lambda x :result(x,True))
rob_data_black['RESULT'] = rob_data_white['RESULT'].apply(lambda x :result(x,False))

In [0]:
def split_transcript(transcript, iswhite=True):
  if iswhite:
    return transcript[::2]
  else:
    return transcript[1::2]

data_white['TRANSCRIPT'] = data_white['TRANSCRIPT_ALL'].apply(lambda x: split_transcript(x,True))
data_black['TRANSCRIPT'] = data_white['TRANSCRIPT_ALL'].apply(lambda x: split_transcript(x,False))

In [0]:
data_white['COLOR'] = 1
data_black['COLOR'] = 0
rob_data_white['COLOR'] = 1
rob_data_black['COLOR'] = 0

In [0]:
data_combine = pd.concat([data_white, data_black], ignore_index=True)
data_rob_combine = pd.concat([rob_data_white, rob_data_black], ignore_index=True)

In [0]:
def generate_reference_time(clock_list, return_game_length=False):
  reference_time = []
  for clock in clock_list:
    hour_minute_second = clock.split(':')
    hour,minute,second = int(hour_minute_second[0]), int(hour_minute_second[1]), int(hour_minute_second[2])
    reference_time.append(3600*hour+minute*60+second)
  if return_game_length:
    try:
      return np.abs(reference_time[0]-reference_time[-1])
    except:
      return np.NaN
  else:
    return reference_time

data_combine['GAME_LENGTH'] = data_combine['CLOCK'].apply(lambda x : generate_reference_time(x,True))
data_combine['CLOCK'] = data_combine['CLOCK'].apply(lambda x : generate_reference_time(x))

In [0]:

data_combine.loc[data_combine['ELO'] >= 2200, 'ELO_RANK'] = 1
data_combine.loc[data_combine['ELO'] < 2200, 'ELO_RANK'] = 2
data_combine.loc[data_combine['ELO'] < 2000, 'ELO_RANK'] = 3
data_combine.loc[data_combine['ELO'] < 1800, 'ELO_RANK'] = 4
data_combine.loc[data_combine['ELO'] < 1600, 'ELO_RANK'] = 5

In [0]:
string2num_dict = {}
def string2num(dataframe):
  column_list = ['GAME_TYPE','OPENING', 'TERMINATION']
  for column in column_list:
    if column == 'TIME_CONTROL' or column == 'TRANSCRIPT' or column == 'TRANSCRIPT_ALL' or column =='RECORD':
      continue
    try:
      dataframe[column].iloc[0][0].isalpha()
      for i,content in enumerate(dataframe[column].unique()):
        dataframe[column] = dataframe[column].replace(content, i+1)
        try:
          string2num_dict[column][content] = i+1
        except:
          string2num_dict[column] = {}
          string2num_dict[column][content] = i+1
    except:
      pass
  return dataframe


data_combine = string2num(data_combine)

In [0]:
data_rob_combine['TERMINATION'] = 1

In [0]:
data_combine['STEPS'] = data_combine['LOCAL_EVAL'].apply(lambda x : len(x))

In [0]:
data_rob_combine['STEPS'] = data_rob_combine['LOCAL_EVAL'].apply(lambda x :len(x))

In [0]:
# How many steps did each material move
def steps_for_material(transcript, King=False, Queen=False, Knight=False, Bishoop=False, Rook=False, Pawn=False):
  king = 0
  queen = 0
  knight = 0
  bishoop = 0
  rook = 0
  steps = len(transcript)
  for t in transcript:
    if 'K' in t:
      king = king + 1
    elif 'Q' in t:
      queen = queen = 1
    elif 'N' in t:
      knight = knight + 1
    elif 'B' in t:
      bishoop = bishoop + 1
    elif 'R' in t:
      rook = rook + 1
  pawn = pawn = steps - king - queen - knight - bishoop - rook

  if King:
    return king
  elif Queen:
    return queen
  elif Knight:
    return knight
  elif Bishoop:
    return bishoop
  elif Rook:
    return rook
  elif Pawn:
    return pawn

In [0]:
data_combine['KING_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, King=True))
data_combine['QUEEN_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Queen=True))
data_combine['KNIGHT_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Knight=True))
data_combine['BISHOP_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Bishoop=True))
data_combine['ROOK_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Rook=True))
data_combine['PAWN_MOVE'] = data_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Pawn=True))


In [0]:
data_rob_combine['KING_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, King=True))
data_rob_combine['QUEEN_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Queen=True))
data_rob_combine['KNIGHT_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Knight=True))
data_rob_combine['BISHOP_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Bishoop=True))
data_rob_combine['ROOK_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Rook=True))
data_rob_combine['PAWN_MOVE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : steps_for_material(x, Pawn=True))


In [0]:
# number of pieces protecting king
def protect_pieces(fianl_position_matrix, iswhite=True):
  # white is represented by capital letters
  # blcak is represented by lowercase letters

  # white
  # find the king first
  king = np.where(fianl_position_matrix=='K')
  king_i, king_j = king[0][0], king[1][0]
  count = 0
  for i in range(king_i,king_i+3):
    for j in range(king_j,king_j+2):
      try:
        if fianl_position_matrix[i][j].isupper():
          count = count + 1
      except:
        # corner benefit
        count = count + 1
  count = count-1

  # black
  # find the king first
  king = np.where(fianl_position_matrix=='k')
  king_i, king_j = king[0][0], king[1][0]
  count = 0
  for i in range(king_i-2,king_i+1):
    for j in range(king_j-1,king_j+1):
      try:
        if fianl_position_matrix[i][j].islower():
          count = count + 1
      except:
        # corner benefit
        count = count +1
  count = count-1
  
  return count

data_combine['SAFE'] = data_combine[data_combine['COLOR']==0]['FINAL_POSITION_MATRIX'].apply(lambda x: protect_pieces(x, False))
data_combine['SAFE'] = data_combine[data_combine['COLOR']==1]['FINAL_POSITION_MATRIX'].apply(lambda x: protect_pieces(x))

In [0]:

data_rob_combine['SAFE'] = data_rob_combine[data_rob_combine['COLOR']==0]['FINAL_POSITION_MATRIX'].apply(lambda x: protect_pieces(x, False))
data_rob_combine['SAFE'] = data_rob_combine[data_rob_combine['COLOR']==1]['FINAL_POSITION_MATRIX'].apply(lambda x: protect_pieces(x))

In [0]:
def claim_draw(result):
  if result == 0.5:
    return 1
  else:
    return 0

data_combine['DRAW'] = data_combine['RESULT'].apply(lambda x:claim_draw(x))

In [0]:
data_rob_combine['DRAW'] = data_rob_combine['RESULT'].apply(lambda x: claim_draw(x))

In [0]:
def castle(transcript, return_king_castle=False):
  king_castle = 0
  queen_castle = 0
  for step,t in enumerate(transcript):
    if t == 'O-O':
      king_castle = step
    elif t == 'O-O-O':
      queen_castle = step
  if return_king_castle:
    return king_castle
  else:
    return queen_castle

data_combine['KING_CASTLE'] = data_combine['TRANSCRIPT'].apply(lambda x : castle(x,True))
data_combine['QUEEN_CASTLE'] = data_combine['TRANSCRIPT'].apply(lambda x : castle(x,False))


In [0]:
data_rob_combine['KING_CASTLE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : castle(x,True))
data_rob_combine['QUEEN_CASTLE'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : castle(x,False))

In [0]:
# pawn promotion number and material
def pawn_promotion(transcript, return_material_weight=False):
  count = 0
  material = 0
  for t in transcript:
    if '=' in t:
      count = count + 1
      if 'Q' in t:
        material = material + 4
      elif 'N' in t:
        material = material + 3
      elif 'R' in t:
        material = material + 2
      elif 'B' in t:
        material = material + 1
  if return_material_weight:
    return material
  else:
    return count

data_combine['PROMOTION_COUNT'] = data_combine['TRANSCRIPT'].apply(lambda x : pawn_promotion(x))
data_combine['PROMOTION_MATERIAL'] = data_combine['TRANSCRIPT'].apply(lambda x : pawn_promotion(x, False))

In [0]:
data_rob_combine['PROMOTION_COUNT'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : pawn_promotion(x))
data_rob_combine['PROMOTION_MATERIAL'] = data_rob_combine['TRANSCRIPT'].apply(lambda x : pawn_promotion(x, False))

In [0]:
def period_LE(local_eval_list, split_type, period, statistic):
  # same split for all game
  if split_type == 'SAME':
    opening = local_eval_list[:9]
    mid = local_eval_list[9:29]
    end = local_eval_list[29:]

  # split game according to steps
  elif split_type == 'ACCORDING':
    mid_start = int(len(local_eval_list) * 9/50)
    mid_end = int(len(local_eval_list) * 29/50)
    opening = local_eval_list[:mid_start]
    mid = local_eval_list[mid_start:mid_end]
    end = local_eval_list[mid_end:]
  
  if period == 'OPENING':
    if statistic == 'MEAN':
      return np.mean(opening)
    elif statistic == 'MODE':
      return stats.mode(opening)
    elif statistic == 'STD':
      return np.std(opening)
  elif period == 'MID':
    if statistic == 'MEAN':
      return np.mean(mid)
    elif statistic == 'MODE':
      return stats.mode(mid)
    elif statistic == 'STD':
      return np.std(mid)
  elif period == 'END':
    if statistic == 'MEAN':
      return np.mean(end)
    elif statistic == 'MODE':
      return stats.mode(end)
    elif statistic == 'STD':
      return np.std(end)

for split_type in ['ACCORDING']:
  for period in ['OPENING','MID','END']:
    for statistic in ['MEAN','MODE','STD']:
      name = 'LE_' + split_type+'_' + period+'_' + statistic
      data_combine[name] = data_combine['LOCAL_EVAL'].apply(lambda x: period_LE(x,split_type, period, statistic))


In [0]:
for split_type in ['ACCORDING']:
  for period in ['OPENING','MID','END']:
    for statistic in ['MEAN','MODE','STD']:
      name = 'LE_' + split_type+'_' + period+'_' + statistic
      data_rob_combine[name] = data_rob_combine['LOCAL_EVAL'].apply(lambda x: period_LE(x,split_type, period, statistic))


In [0]:
def step_time_comsume(referenced_time):
  time = []
  for i in range(len(referenced_time)):
    try:
      time.append(np.abs(referenced_time[i]-referenced_time[i+1]))
    except:
      break
  return time
data_combine['TIME_PER_STEP'] = data_combine['CLOCK'].apply(lambda x: step_time_comsume(x))

In [0]:
def time_analysis(time_per_step, period, statistic):
  time_per_step = [int(t) for t in time_per_step]
  mid_start = int(len(time_per_step) * 9/50)
  mid_end = int(len(time_per_step) * 29/50)
  opening = time_per_step[:mid_start]
  mid = time_per_step[mid_start:mid_end]
  end = time_per_step[mid_end:]

  if period is 'opening':
    if statistic is 'mean':
      return np.mean(opening)
    elif statistic is 'std':
      return np.std(opening)
  elif period is 'mid':
    if statistic is 'mean':
      return np.mean(mid)
    elif statistic is 'std':
      return np.std(mid)
  elif period is 'end':
    if statistic is 'mean':
      return np.mean(end)
    elif statistic is 'std':
      return np.std(end)

for period in ['opening','mid','end']:
  for statistic in ['mean','std']:
    name = "time_{}_{}".format(period,statistic)
    data_rob_combine[name] = data_rob_combine['TIME_PER_STEP'].apply(lambda x: time_analysis(x, period=period, statistic=statistic))
    data_combine[name] = data_combine['TIME_PER_STEP'].apply(lambda x: time_analysis(x, period=period, statistic=statistic))


## Analysis

In [0]:
def num2rank(num):
  num = int(num)
  rank = 0
  if num >= 2200:
    rank = 1
  if num < 2200:
    rank = 2
  if num < 2000:
    rank = 3
  if num < 1800:
    rank = 4
  if num < 1600:
    rank = 5
  return rank

In [0]:
# shuffle data
elo_data = data_combine[['GAME_TYPE', 'OPENING', 'TERMINATION',
       'INCREMENT', 'ALL_TIME', 'CAPTURED', 'Q',
       'N', 'B', 'R', 'P', 'q', 'n', 'b', 'r', 'p', 'check', 'checkmate',
       'insufficient_material', 'repetition', 'stalemate',
       'GAME_LENGTH', 'STEPS', 'KING_MOVE', 'QUEEN_MOVE',
       'KNIGHT_MOVE', 'BISHOP_MOVE', 'ROOK_MOVE', 'PAWN_MOVE', 'DRAW',
       'KING_CASTLE', 'QUEEN_CASTLE', 'PROMOTION_COUNT', 'PROMOTION_MATERIAL',
       'LE_ACCORDING_OPENING_MEAN', 'LE_ACCORDING_OPENING_STD',
       'LE_ACCORDING_MID_MEAN', 'LE_ACCORDING_MID_STD',
       'LE_ACCORDING_END_MEAN', 'LE_ACCORDING_END_STD','FIXED_TIME','ELO',
       'time_opening_mean', 'time_opening_std',
       'time_mid_mean', 'time_mid_std', 'time_end_mean', 'time_end_std']]
elo_data = elo_data.sample(len(data_combine))
for ft in elo_data['FIXED_TIME'].unique():
  # fill nan with mean grouped by fixed time
  elo_data[elo_data['FIXED_TIME']==ft] = elo_data[elo_data['FIXED_TIME']==ft].fillna(elo_data[elo_data['FIXED_TIME']==ft].mean())
elo_data = elo_data.fillna(elo_data.mean())
elo_data['ELO_RANK'] = elo_data['ELO'].apply(lambda x: num2rank(x))

In [0]:
# sns.set()
# plt.figure(figsize=(6,6))
# sns.scatterplot(x="LE_ACCORDING_OPENING_STD", y="ELO", data=elo_data, scatter_kws={"color":"#83919f","alpha":0.1,"s":5})
# plt.show()

sns.set()
plt.figure(figsize=(6,6))
sns.regplot(x="ELO", y="LE_ACCORDING_END_STD", data=elo_data, fit_reg=False, scatter_kws={"color":"#4e4f63","alpha":0.1,"s":5},label="END")
sns.regplot(x="ELO", y="LE_ACCORDING_MID_STD", data=elo_data, fit_reg=False, scatter_kws={"color":"#9dacb1","alpha":0.1,"s":5}, label="MIDDEL")
sns.regplot(x="ELO", y="LE_ACCORDING_OPENING_STD", data=elo_data, fit_reg=False, scatter_kws={"color":"#83919f","alpha":0.1,"s":5}, label="OPENING")
plt.legend()
plt.show()

# Model for elo prediction

In [0]:
def compute_model_evaluation(model_dict, y_true, y_pred, model_type):
  rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  r2 = r2_score(y_true, y_pred)
  accuracy = 0
  for y1,y2 in zip(y_true, y_pred):
    # if num2rank(y1) == num2rank(y2):
    if int(y1) == int(y2):
      accuracy = accuracy + 1
  accuracy = accuracy / len(y_pred)
  print("model",model_type,"rmse: ", rmse, "r2: ", r2, "accuracy: ",accuracy)
  model_dict[model_type] = {'rmse':rmse, 'r2':r2, 'accuracy':accuracy}
  return model_dict


In [0]:
def baseline_model(df):
  model_dict = {}
  model_name = []
  train, val = df.iloc[:int(len(df)*0.7)], df.iloc[int(len(df)*0.7):]
  train_X = train.drop(['ELO','ELO_RANK'], axis=1)
  train_y = train['ELO']
  val_X = val.drop(['ELO','ELO_RANK'], axis=1)
  val_y = val['ELO_RANK']

  # xgb
  xgb = xgboost(n_estimators=500,
                max_depth=10, 
                min_samples_split=2,
                learning_rate=0.003, 
                loss='ls')
  model_name.append('XGB')
  
  # linear regression
  lreg = LinearRegression()
  model_name.append('Linear Regression')

  # ExtraTreesRegressor
  etreg = ExtraTreesRegressor(n_estimators=10, max_features=15, random_state=0)
  model_name.append('ExtraTreesRegressor')

  # ElasticNet
  elnet = ElasticNetCV(alphas=[0.001,0.01,0.1,0.5,1,10], cv=5)
  model_name.append('ElasticNet')

  models1 = [xgb, lreg, etreg, elnet]

  for i,model in enumerate(models1):
    model.fit(train_X, train_y)
    pred_y = model.predict(val_X)
    pred_y = pd.Series(pred_y).apply(lambda x : num2rank(x))
    model_dict = compute_model_evaluation(model_dict, val_y, pred_y, model_name[i])
  return model_dict



In [0]:
def train_model(X_y):
  model_dict = {}
  model_name = []

  h2o.init(max_mem_size = 5) # initializing h2o server
  h2o.remove_all()
  # convert panda data to h2o data
  X_y_h2o = h2o.H2OFrame(X_y)
  # split data
  train_h2o, val_h2o = X_y_h2o.split_frame([0.7])

  ae = H2OAutoEncoderEstimator(activation="Tanh",
                                  hidden=[64,128,256,100,100],
                                  epochs=100,
                                  train_samples_per_iteration=4,
                                  autoencoder=True,
                                  l1=1e-4)
  ae.train(x=train_h2o.drop('ELO_RANK', axis=1).columns, training_frame=train_h2o)
  y_pred = ae.predict(val_h2o.drop(['ELO','ELO_RANK'], axis=1)).as_data_frame()['reconstr_ELO']
  y_pred = pd.Series(y_pred).apply(lambda x : num2rank(x))
  model_dict = compute_model_evaluation(model_dict, val_h2o['ELO_RANK'].as_data_frame()['ELO_RANK'], y_pred, 'autoencoder')

  train_h2o = train_h2o.drop('ELO', axis=1)
  x = train_h2o.drop('ELO_RANK', axis=1).columns
  y = "ELO_RANK"
  val_h2o = val_h2o.drop('ELO', axis=1)
  train_h2o['ELO_RANK'] = train_h2o['ELO_RANK'].asfactor()
  val_X = val_h2o.drop('ELO_RANK', axis=1)
  val_y = val_h2o['ELO_RANK'].as_data_frame()['ELO_RANK']


  nfolds=5
  fold_assignment = 'Modulo'

  dl = H2ODeepLearningEstimator(distribution='bernoulli',
                                  activation='RectifierWithDropout', 
                                  hidden=[100,100,100],
                                  input_dropout_ratio=0.2, 
                                  l1=1e-5, 
                                epochs=10,
                                nfolds=nfolds,
                                fold_assignment=fold_assignment,
                                keep_cross_validation_predictions=True)
  model_name.append('H2ODeepLearning with dropout')

  rf = H2ORandomForestEstimator(#distribution='bernoulli',
                                  ntrees=300,
                                  sample_rate=0.7,
                                  mtries=10, 
                                  nfolds=nfolds,
                                  fold_assignment=fold_assignment,
                                  keep_cross_validation_predictions=True)
  model_name.append('Random Forest with dropout')

  

  models2 = [dl, rf]

  for i,model in enumerate(models2):
    model.train(x=x, y=y, training_frame=train_h2o)
    y_pred = model.predict(val_X)
    y_pred = y_pred.as_data_frame()['predict']
    model_dict = compute_model_evaluation(model_dict, val_y, y_pred, model_name[i])



  return model_dict



#### Baseline Model


In [0]:
fixed_times = sorted(elo_data['FIXED_TIME'].unique())
print(fixed_times)

In [0]:
f600 = elo_data[(elo_data['FIXED_TIME']<=600) & (elo_data['FIXED_TIME']>360)]
baseline_model(f600)
train_model(f600)

In [0]:
f840 = elo_data[(elo_data['FIXED_TIME']<=840) & (elo_data['FIXED_TIME']>600)]
baseline_model(f840)
train_model(f840)

In [0]:
f1020 = elo_data[(elo_data['FIXED_TIME']<=1020) & (elo_data['FIXED_TIME']>840)]
baseline_model(f1020)
train_model(f1020)

In [0]:
f1200 = elo_data[(elo_data['FIXED_TIME']<=1200) & (elo_data['FIXED_TIME']>1020)]
baseline_model(f1200)
train_model(f1200)

In [0]:
f2400 = elo_data[(elo_data['FIXED_TIME']<=2400) & (elo_data['FIXED_TIME']>1200)]
baseline_model(f2400)
train_model(f2400)

In [0]:
f10800 = elo_data[(elo_data['FIXED_TIME']<=10800) & (elo_data['FIXED_TIME']>=2400)]
baseline_model(f10800)
train_model(f10800)

In [0]:
f45 = elo_data[elo_data['FIXED_TIME']<=45]
baseline_model(f45)
train_model(f45)

In [0]:
f120 = elo_data[(elo_data['FIXED_TIME']<=120) & (elo_data['FIXED_TIME']>=60)]
baseline_model(f120)
train_model(f120)

In [0]:
f360 = elo_data[(elo_data['FIXED_TIME']<=360) & (elo_data['FIXED_TIME']>=180)]
baseline_model(f360)
train_model(f360)

In [0]:
baseline_model(elo_data)
train_model(elo_data)

In [0]:
f_0_45 = pd.DataFrame(f45.corr()['ELO'])
f_60_120 = pd.DataFrame(f120.corr()['ELO'])
f_180_360 = pd.DataFrame(f360.corr()['ELO'])
f_120_600 = pd.DataFrame(f600.corr()['ELO'])
f_660_840 = pd.DataFrame(f840.corr()['ELO'])
f_900_1020 = pd.DataFrame(f1020.corr()['ELO'])
f_1080_1200 = pd.DataFrame(f1200.corr()['ELO'])
f_1500_2400 = pd.DataFrame(f2400.corr()['ELO'])
f_2700_5400 = pd.DataFrame(f10800.corr()['ELO'])
corr = pd.concat([f_0_45, f_60_120, f_180_360, f_120_600, f_660_840, f_900_1020, f_1080_1200, f_1500_2400, f_2700_5400],axis=1)

In [0]:
corr.columns = ['f_0_45', 'f_60_120', 'f_180_360', 'f_120_600', 'f_660_840', 'f_900_1020', 'f_1080_1200', 'f_1500_2400', 'f_2700_5400']
corr = corr.T.dropna(axis=1)

In [0]:
plt.figure(figsize=(30,10))
corr_matrix = corr.drop(['CAPTURE','RESULT','RATING_DIFF','SAFE','Q','N','B','R','P','q','n','b','r','p','checkmate','COLOR','FIXED_TIME','ELO'], axis=1)
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
  p2 = sns.heatmap(corr_matrix, vmax=0.4, square=True, cmap="bone_r",annot=True)

In [0]:
corr

#### deep learning model

In [0]:
modle_dict = train_model(elo_data)

## Model for rob test

In [0]:
def baseline_model(df):
  model_dict = {}
  model_name = []
  train, val = df.iloc[:int(len(df)*0.7)], df.iloc[int(len(df)*0.7):]
  train_X = train.drop('ROB', axis=1)
  train_y = train['ROB']
  val_X = val.drop('ROB', axis=1)
  val_y = val['ROB']

  # xgb
  xgb = xgboost(n_estimators=500,
                max_depth=10, 
                min_samples_split=2,
                learning_rate=0.003, 
                loss='ls')
  model_name.append('XGB')
  
  # linear regression
  lreg = LinearRegression()
  model_name.append('Linear Regression')

  # ExtraTreesRegressor
  etreg = ExtraTreesRegressor(n_estimators=10, max_features=15, random_state=0)
  model_name.append('ExtraTreesRegressor')

  # ElasticNet
  elnet = ElasticNetCV(alphas=[0.001,0.01,0.1,0.5,1,10], cv=5)
  model_name.append('ElasticNet')

  models1 = [xgb, lreg, etreg, elnet]

  for i,model in enumerate(models1):
    model.fit(train_X, train_y)
    pred_y = model.predict(val_X)
    pred_y = pd.Series(pred_y).apply(lambda x : num2rank(x))
    model_dict = compute_model_evaluation(model_dict, val_y, pred_y, model_name[i])
  return model_dict



In [0]:
def train_model(X_y):
  model_dict = {}
  model_name = []
  print("begin train model to predict elo rank")

  h2o.init(max_mem_size = 5) # initializing h2o server
  h2o.remove_all()
  # convert panda data to h2o data
  X_y_h2o = h2o.H2OFrame(X_y)
  # split data
  train_h2o, val_h2o = X_y_h2o.split_frame([0.7])

  # h2o data prepare
  x = train_h2o.drop('ROB', axis=1).columns
  y = "ROB"
  train_h2o['ROB'] = train_h2o['ROB'].asfactor()
  val_X = val_h2o.drop('ROB', axis=1)
  val_y = val_h2o['ROB'].as_data_frame()['ROB']

  nfolds=5
  fold_assignment = 'Modulo'

  dl2 = H2ODeepLearningEstimator(distribution='bernoulli',
                                  activation='RectifierWithDropout', 
                                  hidden=[100,100,100],
                                  input_dropout_ratio=0.2, 
                                  l1=1e-5, 
                                epochs=10,
                                nfolds=nfolds,
                                fold_assignment=fold_assignment,
                                keep_cross_validation_predictions=True)
  model_name.append('H2ODeepLearning with dropout')

  rf2 = H2ORandomForestEstimator(#distribution='bernoulli',
                                  ntrees=300,
                                  sample_rate=0.7,
                                  mtries=10, 
                                  nfolds=nfolds,
                                  fold_assignment=fold_assignment,
                                  keep_cross_validation_predictions=True)
  model_name.append('Random Forest with dropout')

  # Edit this list of base models to make different ensembles
  models2 = [dl2, rf2]

  for i,model in enumerate(models2):
    model.train(x=x, y=y, training_frame=train_h2o)
    y_pred = model.predict(val_X)
    y_pred = y_pred.as_data_frame()['predict']
    model_dict = compute_model_evaluation(model_dict, val_y, y_pred, model_name[i])
    model_dict[model_name[i]]['y_pred'] = y_pred
    print("model {} is done".format(model_name[i]))
   

  rmse = []
  r2 = []
  accuracy = []
  max_acc = 0
  for name in model_name:
    rmse.append(model_dict[name]['rmse'])
    r2.append(model_dict[name]['r2'])
    acc = model_dict[name]['accuracy']
    accuracy.append(acc)
    if max_acc < acc:
      max_acc = acc
      choose_model = model_dict[name]
  print("We have choose model {} to predict ELO rank".fotmat(choose_model))

  # plot for elo rank accuracy
  plt.subplot(131)
  plt.plot(model_name, rmse, label=ft)
  plt.legend()
  plt.title("ELO rank rmse")

  plt.subplot(132)
  plt.plot(model_name, r2, label=ft)
  plt.legend()
  plt.title("ELO rank r2")

  plt.subplot(133)
  plt.plot(model_name, accuracy, label=ft)
  plt.legend()
  plt.title("ELO rank accuracy")
  plt.subplots_adjust(left=0, right=2, bottom=0, top=1, wspace=0.5)
  plt.show()

  return model_dict



In [0]:
data_rob_combine['ROB'] = 1
data_combine['ROB'] = 0
rob_test = pd.concat([data_rob_combine, data_combine], join='inner', ignore_index=True)


In [0]:
rob_test = rob_test[['ELO', 'check', 'variant_win',
       'checkmate', 'insufficient_material', 'repetition', 'stalemate',
       'CAPTURE', 'CAPTURED', 'Q', 'N', 'B', 'R', 'P',
       'q', 'n', 'b', 'r', 'p', 
       'TERMINATION', 'STEPS', 'KING_MOVE', 'QUEEN_MOVE', 'KNIGHT_MOVE',
       'BISHOP_MOVE', 'ROOK_MOVE', 'PAWN_MOVE', 'SAFE', 'DRAW', 'KING_CASTLE',
       'QUEEN_CASTLE', 'PROMOTION_COUNT', 'PROMOTION_MATERIAL',
       'LE_ACCORDING_OPENING_MEAN',
       'LE_ACCORDING_OPENING_STD', 'LE_ACCORDING_MID_MEAN', 'LE_ACCORDING_MID_STD',
       'LE_ACCORDING_END_MEAN', 
       'LE_ACCORDING_END_STD', 'ROB','time_opening_mean', 'time_opening_std',
       'time_mid_mean', 'time_mid_std', 'time_end_mean', 'time_end_std']]



In [0]:
# baseline_model(rob_test)
train_model(rob_test)

In [0]:
baseline_model(rob_data)